# python play

In [1]:
for x in [1,2,3,4,5]: # transactions
    for y in "abcde": # txins
        if y == "b":
            break
        print(x, y)

1 a
2 a
3 a
4 a
5 a


# trying out signatures

In [2]:
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256
from Crypto.Signature import DSS

def make_keypair():
    privkey = ECC.generate(curve='P-256')
    pubkey = privkey.public_key()
    return privkey, pubkey 

def make_key(hex):
    key = ECC.import_key(bytes.fromhex(hex))
    return key

def make_hex_key(key):
    binary = key.public_key().export_key(format='DER')
    return binary.hex()

def sign_something(priv_key, something):
    signer = DSS.new(priv_key, 'fips-186-3')
    h = SHA256.new(something)
    signature = signer.sign(h)
    return signature.hex()

def verify_signature(pub_key, something, signature):
    h = SHA256.new(something)
    verifier = DSS.new(pub_key, 'fips-186-3')
    try:
        verifier.verify(h, signature)
        print ("The message is authentic.")
    except ValueError:
        print ("The message is not authentic.")

priv_key, pub_key = make_keypair()
priv_key2, pub_key2 = make_keypair()

# print(pub_key)

message = b'I accept this'
signature = sign_something(priv_key, message)
bitas = bytes.fromhex(signature)
verify_signature(pub_key, message, bitas)

hexkey = make_hex_key(pub_key)

make_key(hexkey)

priv_key.public_key()

The message is authentic.


EccKey(curve='NIST P-256', point_x=103453144897129876028431839281207158881217355283164051232698293403272003163452, point_y=17366669826185541975373305218931516834010948857231229883604105416639207612833)

# trying out transactions

In [3]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"
bob_privkey, bob_pubkey = make_keypair()

# f = open('privatekey.der', 'wt')
# f.write(bob_privkey.export_key(format='DER').hex())
# f.close()

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

unsignedTxIns = bobs_gift_transaction.txins
for uTxIn in unsignedTxIns:
    signature = sign_something(bob_privkey, bobs_gift_transaction.id.encode())
    uTxIn.signature = signature

# Verifying that thing is signed
signa = bytes.fromhex(unsignedTxIns[0].signature)
verify_signature(bob_pubkey, bobs_gift_transaction.id.encode(), signa)

The message is authentic.


# full-node toy model 

todos:
- transaction maken
- block minen
- blockchain checken

In [29]:
from blockchain import *
from transaction import *
from wallet import *
from transaction_pool import *

"""
helper functions
"""
# def make_transaction():




"""
the toy model
"""

# maak mijn wallet aan
toufik_wallet = 'toufiks-wallet'
init_wallet(toufik_wallet)
tfk_priv_key = get_key_from_wallet(toufik_wallet)
tfk_pub_key = tfk_priv_key.public_key()
tfk_address = get_hex_from_key(tfk_pub_key)

# maak een wallet aan voor Bob
bob_wallet = 'bobs-wallet'
init_wallet(bob_wallet)
bob_priv_key = get_key_from_wallet(bob_wallet)
bob_pub_key = bob_priv_key.public_key()
bob_address = get_hex_from_key(bob_pub_key)


# maak de blockchain aan
blockchain = Blockchain()

# maak de unspent transactions list
unspent_txouts = process_transactions(blockchain.chain[0].data, [], 0)

# mine een block
coinbase = get_coinbase_transaction(tfk_address, len(blockchain.chain))
transactions = [coinbase]
blockchain.generate_new_block(transactions)

# maak een transaction

# mine een block

wallet already exists
wallet already exists


{'index': 1, 'hash': '00000430da7eecbdd7997dfe7570cf4fd1f43cb5e05bbc73deb1bb112bde7451', 'previous_hash': '0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208', 'timestamp': 1641142061.3947933, 'proof': 82682, 'data': [{'txins': [{'txout_id': '', 'txout_index': 1, 'signature': ''}], 'txouts': [{'address': '3059301306072a8648ce3d020106082a8648ce3d03010703420004ae1b936f141d4b37355897e82f86afe219e0c2dce78f040954a9ace5d6cbeeb8277ea923e75f1ed234b22286d43c9134689f53a90662465b531436adf0be2484', 'amount': 50}], 'id': '746ccd31d2b4782d81b2d71a4f93f657d082c294d6d33ed7a8c05cd3d150a9a0'}]}